In [25]:
# all 45 people, 7 day of data, 30 min rolling average, larger spheres, no secondary grids

In [26]:
from scipy.io import loadmat
from scipy.stats import zscore
from decimal import *
import pandas as pd
import numpy as np
import datetime as dt
import os
import sys
import math
import pandas as pd
import numpy as np
from pprint import pprint
import json
import requests
import datetime
from datetime import date
import os

In [27]:
pubdata = loadmat('PubData.mat', struct_as_record = True, squeeze_me = True)

In [28]:
pubdata.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Cases50'])

In [29]:
pubdata = {k:v for k, v in pubdata.items() if k[0] != '_'}

In [30]:
# duration of symptoms
a = pubdata['Cases50'][0,1]
b = pubdata['Cases50'][4,1]
c = pubdata['Cases50'][2,1]
d = pubdata['Cases50'][5,1]
#print(a, b, c, d)

#for i in range(0, 50, 1):
#    dtion = pubdata['Cases50'][i,1]
#    print(str('Duration of person ') + str(i) + str(': ') + str(dtion))

### Version info

In [31]:
#v1: all 45 peoples data, plus raw data rods and timing
#v2: added rolling mean, every 10 minutes 
#v3: changed rolling mean to every 30 minutes

In [32]:
# person variable for file names
person = str("pALL45")

# version variable for file name
version = "v2.0"

# viz type: 2 days, 2 weeks, or 4 weeks
viztype = "7d_zgridbox"

In [33]:
cols = ('temp', 'hr', 'hrv', 'breath')

In [34]:
# creates a list of df's with data from each person
# selects only the 42nd to 49th day of data from each dataframe
# note: for i = 1, 3, 11, 24, 41 - they have less than 10080 data points

df_c = []
person_list = [0, 2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]

for i in person_list:
    df = pd.DataFrame(pubdata['Cases50'][i,0], columns = cols)
    df = df.iloc[60480:70560,:]
    df['temp'] = df['temp'].rolling(30).mean()
    df['hr'] = df['hr'].rolling(30).mean()
    df['hrv'] = df['hrv'].rolling(30).mean()
    df['breath'] = df['breath'].rolling(30).mean()
    df = df.iloc[::30,:]
    df = df.iloc[1:]
    df_c.append(df)
    

In [35]:
def make_interpolater(left_min, left_max, right_min, right_max): 
    # Figure out how 'wide' each range is  
    leftSpan = left_max - left_min  
    rightSpan = right_max - right_min  

    # Compute the scale factor between left and right values 
    scaleFactor = float(rightSpan) / float(leftSpan) 

    # create interpolation function using pre-calculated scaleFactor
    def interp_fn(value):
        return right_min + (value-left_min)*scaleFactor


    return interp_fn

In [36]:
# for loop to run z scores, scale the data, drop unnecessary columns
# append the final results for each person's data, a list, to a list of lists: big_list

big_list = []

for df in df_c:
    df[['hr_zscore', 'hrv_zscore']] = df[['hr', 'hrv']].apply(zscore)
    
    col_min = df['hr_zscore'].min() 
    col_max = df['hr_zscore'].max() 
    col_list = df['hr_zscore'].tolist()
    scalar = make_interpolater(col_min + 0.000002, col_max, -200, 200) 
    col_interp = [scalar(x) for x in col_list] 
    df['hr_zscale'] = col_interp
    
    col_mn = df['hrv_zscore'].min() 
    col_mx = df['hrv_zscore'].max() 
    col_lst = df['hrv_zscore'].tolist()
    scalar = make_interpolater(col_mn + 0.000002, col_mx, -200, 200) 
    col_intrp = [scalar(x) for x in col_lst] 
    df['hrv_zscale'] = col_intrp
      
    col_mn = df['temp'].min() 
    col_mx = df['temp'].max() 
    col_lst = df['temp'].tolist()
    scalar = make_interpolater(col_mn + 0.000002, col_mx, 5, 20) 
    col_intrp = [scalar(x) for x in col_lst] 
    df['temp_scale'] = col_intrp
    
    col_mn = df['hr'].min() 
    col_mx = df['hr'].max() 
    col_lst = df['hr'].tolist()
    scalar = make_interpolater(col_mn + 0.000002, col_mx, 5, 20) 
    col_intrp = [scalar(x) for x in col_lst] 
    df['hr_scale'] = col_intrp
    
    col_mn = df['hrv'].min() 
    col_mx = df['hrv'].max() 
    col_lst = df['hrv'].tolist()
    scalar = make_interpolater(col_mn + 0.000002, col_mx, 5, 20) 
    col_intrp = [scalar(x) for x in col_lst] 
    df['hrv_scale'] = col_intrp
    
    col_mn = df['breath'].min() 
    col_mx = df['breath'].max() 
    col_lst = df['breath'].tolist()
    scalar = make_interpolater(col_mn + 0.000002, col_mx, 5, 20) 
    col_intrp = [scalar(x) for x in col_lst] 
    df['b_scale'] = col_intrp
    
    df = df.drop(['hr', 'hrv', 'hr_zscore', 'hrv_zscore', 'breath'], axis=1)
    
    zrange = len(df.index)
    zr = (250/zrange)
    z = np.arange(1.5, 251.5, zr)
    df['timeaxis'] = z
    
    full = []
    for col in df:
        a = df[col].values.tolist()
        full.append(a)

        listxx = [item for sublist in full for item in sublist]

    big_list.append(listxx)


In [37]:
new_df = pd.DataFrame(big_list)
print(new_df.shape)

(45, 2680)


In [38]:
print(len(big_list[0]))

2680


In [39]:
day1 = 60480
daymax = 70560
varmax = int(((daymax - day1)/30)-1)
ltrial = list(range(0, varmax, 1))
tfunc = [('t_{}'.format(x+1)) for x in ltrial]
hrfunc = [('hr_zscale_{}'.format(x+1)) for x in ltrial]
hrvfunc = [('hrv_zscale_{}'.format(x+1)) for x in ltrial]
tsfunc = [('t_scale_{}'.format(x+1)) for x in ltrial]
hrsfunc = [('hr_scale_{}'.format(x+1)) for x in ltrial]
hrvsfunc = [('hrv_scale_{}'.format(x+1)) for x in ltrial]
bsfunc = [('b_scale_{}'.format(x+1)) for x in ltrial]
bfunc = [('time_{}'.format(x+1)) for x in ltrial]

In [40]:
colnamelist = tfunc + hrfunc + hrvfunc + tsfunc + hrsfunc + hrvsfunc + bsfunc + bfunc

In [41]:
len(colnamelist)

2680

In [42]:
new_df.columns = colnamelist

In [43]:
xroot = (list(range(-288, 324, 72))) * 5
yroot = (list(range(-162, 198, 72))) * 9
zroot = 0

In [44]:
new_df['xroot'] = xroot
new_df['yroot'] = yroot
new_df['zroot'] = zroot

In [45]:
# set DEBUG flag
DEBUG = 0

# Initialize all ANTz Node variables
# You shouldn't need to change anything in this section
# you can find an explanation of these terms in Appendix B of the user manual
id = 0
type = 5
data = id
selected = 0
parent_id = 0
branch_level = 0
child_id = id
child_index = 0
palette_id = 0
ch_input_id = 0
ch_output_id = 0
ch_last_updated = 0
average = 0
samples = 0
aux_a_x = 0
aux_a_y = 0
aux_a_z = 0
aux_b_x = 0
aux_b_y = 0
aux_b_z = 0
color_shift = 0
rotate_vec_x = 0
rotate_vec_y = 0
rotate_vec_z = 0
rotate_vec_s = 1
scale_x = 0
scale_y = 0
scale_z = 0
translate_x = 0
translate_y = 0
translate_z = 0
tag_offset_x = 0
tag_offset_y = 0
tag_offset_z = 0
rotate_rate_x = 0
rotate_rate_y = 0
rotate_rate_z = 0
rotate_x = 0
rotate_y = 0
rotate_z = 0
scale_rate_x = 0
scale_rate_y = 0
scale_rate_z = 0
translate_rate_x = 0
translate_rate_y = 0
translate_rate_z = 0
translate_vec_x = 0
translate_vec_y = 0
translate_vec_z = 0
shader = 0
geometry = 7
line_width = 1
point_size = 0
ratio = 0.1
color_index = 0
color_r = 110
color_g = 110
color_b = 110
color_a = 255
color_fade = 0
texture_id = 0
hide = 0
freeze = 0
topo = 3
facet = 0
auto_zoom_x = 0
auto_zoom_y = 0
auto_zoom_z = 0
trigger_hi_x = 0
trigger_hi_y = 0
trigger_hi_z = 0
trigger_lo_x = 0
trigger_lo_y = 0
trigger_lo_z = 1
set_hi_x = 0
set_hi_y = 0
set_hi_z = 0
set_lo_x = 0
set_lo_y = 0
set_lo_z = 0
proximity_x = 0
proximity_y = 0
proximity_z = 0
proximity_mode_x = 0
proximity_mode_y = 0
proximity_mode_z = 0
segments_x = 16
segments_y = 16
segments_z = 0
tag_mode = 0
format_id = 0
table_id = 0
record_id = id
size = 420

In [46]:
# Person 1 node files
pnode = str(person) + "_" + "temppredict_" + str(viztype) + "_" + str(version) + "_node.csv"
# Person 1 tag files
ptag = str(person) + "_" + "temppredict_" + str(viztype) + "_" + str(version) + "_tag.csv"

## Titles of files

### Change name of dataframe to match which viz version is being run

In [47]:
# Open ANTz Node file for writing
fout = open(pnode,"w")

# Open the Tag file to add metadata to ANTz objects
ftag = open(ptag,"w")
# Write the header string
outputstring = "id,record_id,table_id,title,description\n"
ftag.write(outputstring)

# Write rows for header, world, camera views (4), and grid to Node file
outputstring = "id,type,data,selected,parent_id,branch_level,child_id,child_index,palette_id,ch_input_id,ch_output_id,ch_last_updated,average,samples,aux_a_x,aux_a_y,aux_a_z,aux_b_x,aux_b_y,aux_b_z,color_shift,rotate_vec_x,rotate_vec_y,rotate_vec_z,rotate_vec_s,scale_x,scale_y,scale_z,translate_x,translate_y,translate_z,tag_offset_x,tag_offset_y,tag_offset_z,rotate_rate_x,rotate_rate_y,rotate_rate_z,rotate_x,rotate_y,rotate_z,scale_rate_x,scale_rate_y,scale_rate_z,translate_rate_x,translate_rate_y,translate_rate_z,translate_vec_x,translate_vec_y,translate_vec_z,shader,geometry,line_width,point_size,ratio,color_index,color_r,color_g,color_b,color_a,color_fade,texture_id,hide,freeze,topo,facet,auto_zoom_x,auto_zoom_y,auto_zoom_z,trigger_hi_x,trigger_hi_y,trigger_hi_z,trigger_lo_x,trigger_lo_y,trigger_lo_z,set_hi_x,set_hi_y,set_hi_z,set_lo_x,set_lo_y,set_lo_z,proximity_x,proximity_y,proximity_z,proximity_mode_x,proximity_mode_y,proximity_mode_z,segments_x,segments_y,segments_z,tag_mode,format_id,table_id,record_id,size\n"
fout.write(outputstring)

# Row for world parameters
outputstring = "1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for first camera view
outputstring = "2,1,2,0,0,0,2,2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0.008645,0.825266,-0.564678,1,1,1,-32.446629,-180.908295,143.514175,0,0,1,0,0,0,55.620094,0.600200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,214.306686,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Row for second camera view
outputstring = "3,1,3,0,2,1,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,-1,1,1,1,-0.500000,0,571.750000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Third camera view
outputstring = "4,1,4,0,2,1,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,-0,1,1,1,0,-90,7,0,0,1,0,0,0,90,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Fourth camera view
outputstring = "5,1,5,0,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1,-0,-0,1,1,1,85,0,7,0,0,1,0,0,0,90,270,0,0,0,0,0,0,0,-0,0,0,0,0,1,0,0.1,0,50,101,101,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,16,0,0,0,0,0,420\n"
fout.write(outputstring)
# Default Grid
outputstring = "6,6,6,1,0,0,0,1,0,0,0,0,0,1,400,400,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.1,3,0,0,255,150,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1, 1,0,0,0,0,0,420\n"
fout.write(outputstring)

# Initialize increments
taginc = 0
id = 6

numlevel0toroids = 1
numlevel1toroids = 1

for index, row in new_df.iterrows() :
    id += 1
    data = id
    type = 5
    parent_id = 0
    selected = 0
    branch_level = 0
    child_id = id
    scale_x = 1
    scale_y = scale_x
    scale_z = scale_x 
    translate_x = row['xroot']
    translate_y = row['yroot']
    translate_z = row['zroot']
    rotate_x = 0
    rotate_y = 0
    rotate_z = 0
    ratio = .1
    line_width= 1
    geometry = 16
    topo = 5
    color_r = 24
    color_g = 123
    color_b = 205
    color_a = 255
    record_id = id 
    texture_id= 0

    # Creat output string and write to ANTz Node File
    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
    fout.write(outputstring)

    # Output to Tag file
    tagtext = "Person: " + str(index)
    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
    ftag.write(tagstring)
    taginc += 1

    level0objectid = id
    # grid object
    for j in range(numlevel1toroids):    
        id += 1
        data = id
        type = 5
        parent_id = level0objectid
        selected = 0
        branch_level = 1
        child_id = id
        scale_x = .08
        scale_y = scale_x
        scale_z = scale_x 
        translate_x = 0
        translate_y = 0
        translate_z = 0
        rotate_x = 0
        rotate_y = 0
        rotate_z = 0
        ratio = .1
        line_width= 2
        geometry = 20
        topo = 8
        color_r = 24
        color_g = 123
        color_b = 205
        color_a = 255
        record_id = id 
        segments_x = 1
        segments_y = 1
        texture_id = 2
        aux_a_x = 400
        aux_a_y = 400
        aux_a_ = 0

        # Creat output string and write to ANTz Node File
        outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
        fout.write(outputstring)

        # Output to Tag file
        tagtext = ""
        tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
        ftag.write(tagstring)
        taginc += 1

        level1objectid = id
        numlevel2toroids = 335
        
        for k in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            column = str(str("hrv_zscale_") + str(int(k)+1))
            translate_x = row[column]
            col = str(str("hr_zscale_") + str(int(k)+1))
            translate_y = row[col]
            colm = str(str("time_") + str(int(k)+1))
            translate_z = row[colm]
            scale_x = 4
            scale_y = scale_x
            scale_z = scale_x
            ratio = .1
            line_width= 2
            geometry = 3
            topo = 2
            colmn = str(str("t_") + str(int(k)+1))
            if row[colmn] <= 35:
                color_r = 27
                color_g = 219
                color_b = 27
            elif (row[colmn] > 35) & (row[colmn] <= 35.5):
                color_r = 26
                color_g = 219
                color_b = 155
            elif (row[colmn] > 35.5) & (row[colmn] <= 36):
                color_r = 35
                color_g = 156
                color_b = 219
            elif (row[colmn] > 36) & (row[colmn] <= 36.5):
                color_r = 40
                color_g = 63
                color_b = 219
            elif (row[colmn] > 36.5) & (row[colmn] <= 37):
                color_r = 74
                color_g = 40
                color_b = 219
            elif (row[colmn] > 37) & (row[colmn] <= 37.5):
                color_r = 121
                color_g = 39
                color_b = 219
            elif (row[colmn] > 37.5) & (row[colmn] <= 38):
                color_r = 219
                color_g = 26
                color_b = 213
            elif (row[colmn] > 38) & (row[colmn] <= 38.5):
                color_r = 219
                color_g = 16
                color_b = 51
            elif (row[colmn] > 38.5) & (row[colmn] <= 39):
                color_r = 219
                color_g = 58
                color_b = 24
            elif (row[colmn] > 39) & (row[colmn] <= 39.5):
                color_r = 219
                color_g = 105
                color_b = 29
            elif (row[colmn] > 39.5) & (row[colmn] <= 40):
                color_r = 219
                color_g = 140
                color_b = 26
            elif (row[colmn] >= 40):
                color_r = 219
                color_g = 169
                color_b = 25
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            # Output to Tag file
            tagtext = str("Time: ") + str(index) + str("// Temp: ") + str(row[colmn])
            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
            ftag.write(tagstring)
            taginc += 1

        numlevel2toroids = 1
        
        # time rod
        for m in range(numlevel2toroids):
            id += 1
            data = id
            type = 5
            parent_id = level1objectid
            selected = 0
            branch_level = 2
            child_id = id
            scale_x = .37
            scale_y = scale_x
            scale_z = scale_x 
            translate_x = -200
            translate_y = -200
            translate_z = 1.5
            rotate_x = 0
            rotate_y = 0
            rotate_z = 0
            ratio = .1
            line_width= 1
            geometry = 7
            topo = 3
            color_r = 12
            color_g = 242
            color_b = 143
            color_a = 255
            record_id = id 
            texture_id= 0

            # Creat output string and write to ANTz Node File
            outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
            fout.write(outputstring)

            # Output to Tag file
            tagtext = ""
            tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
            ftag.write(tagstring)
            taginc += 1

            level2objectid = id
            numlevel3toroids = 1
            for n in range(numlevel3toroids):    
                id += 1
                data = id
                type = 5
                parent_id = level2objectid
                selected = 0
                branch_level = 3
                child_id = id
                scale_x = 6
                scale_y = scale_x
                scale_z = 226.5 
                translate_x = 0
                translate_y = 0
                translate_z = 0
                rotate_x = 90
                rotate_y = 0
                rotate_z = 0
                ratio = .1
                line_width= 1
                geometry = 19
                topo = 6
                color_r = 191
                color_g = 191
                color_b = 191
                color_a = 200
                record_id = id 
                texture_id= 0

                # Creat output string and write to ANTz Node File
                outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                fout.write(outputstring)

                # Output to Tag file
                tagtext = ""
                tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                ftag.write(tagstring)
                taginc += 1
                
                level3objectid = id
                numlevel4toroids = 7
                # time markers
                for lt in range(numlevel4toroids):
                    id += 1
                    data = id
                    type = 5
                    parent_id = level3objectid
                    selected = 0
                    branch_level = 4
                    child_id = id
                    scale_x = 25
                    scale_y = 10
                    scale_z = 3 
                    translate_x = -180 + (.54*lt)
                    translate_y = 0
                    translate_z = 0
                    rotate_x = 0
                    rotate_y = 90
                    rotate_z = 0
                    ratio = .1
                    line_width= 1
                    geometry = 11
                    topo = 2
                    color_r = 107
                    color_g = 196
                    color_b = 219
                    color_a = 255
                    record_id = id 

                    # Creat output string and write to ANTz Node File
                    outputstring = str(id) + "," + str(type) + "," + str(data) + "," + str(selected) + "," + str(parent_id) + "," + str(branch_level) + "," + str(child_id) + "," + str(child_index) + "," + str(palette_id) + "," + str(ch_input_id) + "," + str(ch_output_id) + "," + str(ch_last_updated) + "," + str(average) + "," + str(samples) + "," + str(aux_a_x) + "," + str(aux_a_y) + "," + str(aux_a_z) + "," + str(aux_b_x) + "," + str(aux_b_y) + "," + str(aux_b_z) + "," + str(color_shift) + "," + str(rotate_vec_x) + "," + str(rotate_vec_y) + "," + str(rotate_vec_z) + "," + str(rotate_vec_s) + "," + str(scale_x) + "," + str(scale_y) + "," + str(scale_z) + "," + str(translate_x) + "," + str(translate_y) + "," + str(translate_z) + "," + str(tag_offset_x) + "," + str(tag_offset_y) + "," + str(tag_offset_z) + "," + str(rotate_rate_x) + "," + str(rotate_rate_y) + "," + str(rotate_rate_z) + "," + str(rotate_x) + "," + str(rotate_y) + "," + str(rotate_z) + "," + str(scale_rate_x) + "," + str(scale_rate_y) + "," + str(scale_rate_z) + "," + str(translate_rate_x) + "," + str(translate_rate_y) + "," + str(translate_rate_z) + "," + str(translate_vec_x) + "," + str(translate_vec_y) + "," + str(translate_vec_z) + "," + str(shader) + "," + str(geometry) + "," + str(line_width) + "," + str(point_size) + "," + str(ratio) + "," + str(color_index) + "," + str(color_r) + "," + str(color_g) + "," + str(color_b) + "," + str(color_a) + "," + str(color_fade) + "," + str(texture_id) + "," + str(hide) + "," + str(freeze) + "," + str(topo) + "," + str(facet) + "," + str(auto_zoom_x) + "," + str(auto_zoom_y) + "," + str(auto_zoom_z) + "," + str(trigger_hi_x) + "," + str(trigger_hi_y) + "," + str(trigger_hi_z) + "," + str(trigger_lo_x) + "," + str(trigger_lo_y) + "," + str(trigger_lo_z) + "," + str(set_hi_x) + "," + str(set_hi_y) + "," + str(set_hi_z) + "," + str(set_lo_x) + "," + str(set_lo_y) + "," + str(set_lo_z) + "," + str(proximity_x) + "," + str(proximity_y) + "," + str(proximity_z) + "," + str(proximity_mode_x) + "," + str(proximity_mode_y) + "," + str(proximity_mode_z) + "," + str(segments_x) + "," + str(segments_y) + "," + str(segments_z) + "," + str(tag_mode) + "," + str(format_id) + "," + str(table_id) + "," + str(record_id) + "," + str(size) + "\n"
                    fout.write(outputstring)

                    # Output to Tag file
                    tagtext = str("Day: ") + str(lt)
                    tagstring = str(taginc) + "," + str(record_id) + ",0,\"" + tagtext + "\",\"\"\n"
                    ftag.write(tagstring)
                    taginc += 1        
            
  
fout.close()